In [1]:
import os
import shutil

data_path = 'chest_xray_dataset'
combined_path = 'dataset'
os.makedirs(combined_path, exist_ok=True)

# Combine all images from train, val, and test into a single directory for each class
for split in ['train', 'val', 'test']:
    for class_name in ['NORMAL', 'PNEUMONIA']:
        source_dir = os.path.join(data_path, split, class_name)
        target_dir = os.path.join(combined_path, class_name)
        os.makedirs(target_dir, exist_ok=True)
        
        # Check if source directory exists
        if not os.path.exists(source_dir):
            print(f"Source directory {source_dir} does not exist. Skipping.")
            continue
        
        for file_name in os.listdir(source_dir):
            source_file = os.path.join(source_dir, file_name)
            target_file = os.path.join(target_dir, file_name)
            
            # Check if the file already exists in the target directory to avoid overwriting
            if os.path.exists(target_file):
                print(f"File {target_file} already exists. Skipping.")
                continue
            
            try:
                shutil.copyfile(source_file, target_file)
            except Exception as e:
                print(f"Failed to copy {source_file} to {target_file}: {e}")


Failed to copy chest_xray_dataset/train/PNEUMONIA/.ipynb_checkpoints to dataset/PNEUMONIA/.ipynb_checkpoints: [Errno 21] Is a directory: 'chest_xray_dataset/train/PNEUMONIA/.ipynb_checkpoints'


In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset

# Define transformations
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert images to grayscale if not already
    transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize the images
])


# Load the combined dataset
combined_dataset = datasets.ImageFolder(combined_path, transform=transform)

# Separate NORMAL and PNEUMONIA datasets
normal_indices = [i for i, (img, label) in enumerate(combined_dataset) if label == combined_dataset.class_to_idx['NORMAL']]
pneumonia_indices = [i for i, (img, label) in enumerate(combined_dataset) if label == combined_dataset.class_to_idx['PNEUMONIA']]

normal_dataset = Subset(combined_dataset, normal_indices)
pneumonia_dataset = Subset(combined_dataset, pneumonia_indices)

# Create data loaders for each class
batch_size = 32
normal_loader = DataLoader(normal_dataset, batch_size=batch_size, shuffle=True)
pneumonia_loader = DataLoader(pneumonia_dataset, batch_size=batch_size, shuffle=True)

In [3]:
import torch.nn as nn

# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Flatten(),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

netG = Generator().to(device)
netD = Discriminator().to(device)

criterion = nn.BCELoss()

fixed_noise = torch.randn(64, 100, 1, 1, device=device)
real_label = 1.
fake_label = 0.

optimizerD = torch.optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [5]:
import torch
import torchvision.utils as vutils
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import os

save_dir = 'NORMAL'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

num_epochs = 20

for epoch in range(num_epochs):
    for i, data in enumerate(normal_loader, 0):
        # Concatenate list of tensors into a single tensor
        real_images, _ = data
        real_images = real_images.to(device)
        # Train Discriminator with real batch
        netD.zero_grad()
        batch_size = real_images.size(0)
        labels = torch.full((batch_size,), real_label, dtype=torch.float, device=device)
        output = netD(real_images).mean(1)

        errD_real = criterion(output, labels)
        errD_real.backward()
        D_x = output.mean().item()

        # Train Discriminator with fake batch
        noise = torch.randn(batch_size, 100, 1, 1, device=device)
        fake_images = netG(noise)
        labels.fill_(fake_label)
        output = netD(fake_images.detach()).mean(1)
        errD_fake = criterion(output, labels)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        # Train Generator
        netG.zero_grad()
        labels.fill_(real_label)
        output = netD(fake_images).mean(1)
        errG = criterion(output, labels)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # Print training stats
        if i % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}] Step [{i}/{len(normal_loader)}] Loss_D: {errD.item()} Loss_G: {errG.item()} D(x): {D_x} D(G(z)): {D_G_z1}/{D_G_z2}')

    # Save generated images for every epoch
    with torch.no_grad():
        fake_images = netG(fixed_noise).detach().cpu()
        grid = vutils.make_grid(fake_images, padding=2, normalize=True)
        plt.imshow(grid.permute(1, 2, 0), cmap='gray')
        plt.show()
        
        # Save images to folder
        for idx, image in enumerate(fake_images):
            save_path = os.path.join(save_dir, f'epoch_{epoch+1}_image_{idx+1}.png')
            save_image(image, save_path)

Epoch [1/20] Step [0/50] Loss_D: 1.4386096000671387 Loss_G: 4.743978500366211 D(x): 0.5296969413757324 D(G(z)): 0.54376220703125/0.009061941877007484
Epoch [2/20] Step [0/50] Loss_D: 0.008851748891174793 Loss_G: 7.3370041847229 D(x): 0.9937028288841248 D(G(z)): 0.002529300982132554/0.0006698048673570156
Epoch [3/20] Step [0/50] Loss_D: 0.0037087153177708387 Loss_G: 7.968418121337891 D(x): 0.9974547624588013 D(G(z)): 0.0011592459632083774/0.00036303841625340283
Epoch [4/20] Step [0/50] Loss_D: 0.0024849469773471355 Loss_G: 8.21082878112793 D(x): 0.9982721209526062 D(G(z)): 0.0007549664005637169/0.0002921766717918217
Epoch [5/20] Step [0/50] Loss_D: 0.0019377145217731595 Loss_G: 8.05799674987793 D(x): 0.9987524151802063 D(G(z)): 0.0006889909273013473/0.00035352923441678286
Epoch [6/20] Step [0/50] Loss_D: 0.0012646547984331846 Loss_G: 8.537962913513184 D(x): 0.9990653991699219 D(G(z)): 0.0003294643247500062/0.00021172550623305142
Epoch [7/20] Step [0/50] Loss_D: 0.0008603736641816795 Los

In [6]:
import torchvision.utils as vutils
import matplotlib.pyplot as plt

save_dir = 'PNEUMONIA'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

num_epochs = 20

for epoch in range(num_epochs):
    for i, data in enumerate(pneumonia_loader, 0):
        # Concatenate list of tensors into a single tensor
        real_images, _ = data
        real_images = real_images.to(device)
        # Train Discriminator with real batch
        netD.zero_grad()
        batch_size = real_images.size(0)
        labels = torch.full((batch_size,), real_label, dtype=torch.float, device=device)
        output = netD(real_images).mean(1)

        errD_real = criterion(output, labels)
        errD_real.backward()
        D_x = output.mean().item()

        # Train Discriminator with fake batch
        noise = torch.randn(batch_size, 100, 1, 1, device=device)
        fake_images = netG(noise)
        labels.fill_(fake_label)
        output = netD(fake_images.detach()).mean(1)
        errD_fake = criterion(output, labels)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        # Train Generator
        netG.zero_grad()
        labels.fill_(real_label)
        output = netD(fake_images).mean(1)
        errG = criterion(output, labels)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # Print training stats
        if i % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}] Step [{i}/{len(pneumonia_loader)}] Loss_D: {errD.item()} Loss_G: {errG.item()} D(x): {D_x} D(G(z)): {D_G_z1}/{D_G_z2}')

    # Save generated images for every epoch
    with torch.no_grad():
        fake_images = netG(fixed_noise).detach().cpu()
        grid = vutils.make_grid(fake_images, padding=2, normalize=True)
        plt.imshow(grid.permute(1, 2, 0), cmap='gray')
        plt.show()
        
        # Save images to folder
        for idx, image in enumerate(fake_images):
            save_path = os.path.join(save_dir, f'epoch_{epoch+1}_image_{idx+1}.png')
            save_image(image, save_path)

Epoch [1/20] Step [0/134] Loss_D: 9.413693624082953e-05 Loss_G: 10.29275131225586 D(x): 0.9999514818191528 D(G(z)): 4.556115163723007e-05/3.6096604162594303e-05
Epoch [1/20] Step [50/134] Loss_D: 6.797494279453531e-05 Loss_G: 10.678693771362305 D(x): 0.99996018409729 D(G(z)): 2.8111247956985608e-05/2.4465041860821657e-05
Epoch [1/20] Step [100/134] Loss_D: 6.807357567595318e-05 Loss_G: 10.558014869689941 D(x): 0.9999641180038452 D(G(z)): 3.215175092918798e-05/2.731109270825982e-05
Epoch [2/20] Step [0/134] Loss_D: 6.109396053943783e-05 Loss_G: 10.523611068725586 D(x): 0.9999715089797974 D(G(z)): 3.2563657441642135e-05/2.8285816370043904e-05
Epoch [2/20] Step [50/134] Loss_D: 7.023988291621208e-05 Loss_G: 10.369918823242188 D(x): 0.9999679327011108 D(G(z)): 3.813865623669699e-05/3.219324571546167e-05
Epoch [2/20] Step [100/134] Loss_D: 5.532328214030713e-05 Loss_G: 10.721857070922852 D(x): 0.9999710917472839 D(G(z)): 2.640854836499784e-05/2.2777199774282053e-05
Epoch [3/20] Step [0/134]